## Kaggle Competition : Dogs vs Cats Image classification

Preprocess our images (collect the VGG16 - FNN outputs) and save their bottleneck features

In [110]:
import numpy as np
import datetime
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16


# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'

train_data_dir = 'dogs-vs-cats-redux-kernels-edition/train' #'Lab/data/DogCatChallenge/sampleDeep/train'
validation_data_dir = 'dogs-vs-cats-redux-kernels-edition/valid' #'Lab/data/DogCatChallenge/sampleDeep/valid'
#test_data_dir = 'C:\Users\adrie\OneDrive\Bureau\Informatique\Projets\CV\DTY Lab\dogs-vs-cats\test1\test1'

# Function that instanciates the convolutional part of the VGG16 pre-trained model on Imagenet and that runs it on our training and validation data

def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build and load the VGG16 network without the fully connected layers
    model = VGG16(
        include_top=False, #remove the top fully-connected NN
        weights="imagenet")

    # preparation of the training data
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, # this means our generator will only yield batches of data, no labels
        shuffle=True,
        seed=42) # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data
    
    # Generation of the predictions for the input samples from the training data generator and return them as a numpy array that we can saved
    bottleneck_features_train = model.predict(generator)  #nb_train_samples#
    # save the output as a Numpy array
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    # preparation of the validation data
    
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        #batch_size=batch_size,######
        class_mode=None,
        shuffle=True,
        seed=42)
    
    # Generation of the predictions for the input samples from the validation data generator and return them as a numpy array that we can saved
    bottleneck_features_validation = model.predict(generator) #nb_validation_samples#
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

    
# Function that trains a small fully-connected model on top of the stored previous features

save_bottleneck_features()

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


Train our added FNN model

In [111]:
nb_train_samples = 20000
nb_validation_samples = 5000
epochs = 50
batch_size = 16

def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    # Building of the small fully-connected model
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    # Configuration of the learning process
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    # Training of the model
    history = model.fit(train_data,
            train_labels, 
            epochs=epochs,
            batch_size=batch_size, 
            steps_per_epoch = 20, ################
            validation_data=(validation_data,validation_labels), 
            #callbacks=[tensorboard_callback],
            verbose=2)
    
    model.save('bottleneck_fc_model') 

train_top_model()
# %reload_ext tensorboard
# %tensorboard --logdir logs

Epoch 1/50
20/20 - 4s - loss: 1.6056 - accuracy: 0.5906 - val_loss: 1.2555 - val_accuracy: 0.5336
Epoch 2/50
20/20 - 2s - loss: 0.6146 - accuracy: 0.6938 - val_loss: 0.5976 - val_accuracy: 0.6914
Epoch 3/50
20/20 - 3s - loss: 0.4957 - accuracy: 0.7500 - val_loss: 0.3366 - val_accuracy: 0.8550
Epoch 4/50
20/20 - 2s - loss: 0.4333 - accuracy: 0.8219 - val_loss: 0.3405 - val_accuracy: 0.8594
Epoch 5/50
20/20 - 2s - loss: 0.4512 - accuracy: 0.7844 - val_loss: 0.3287 - val_accuracy: 0.8662
Epoch 6/50
20/20 - 2s - loss: 0.3982 - accuracy: 0.8594 - val_loss: 0.3425 - val_accuracy: 0.8512
Epoch 7/50
20/20 - 2s - loss: 0.3836 - accuracy: 0.8625 - val_loss: 1.1050 - val_accuracy: 0.6350
Epoch 8/50
20/20 - 2s - loss: 0.3954 - accuracy: 0.8344 - val_loss: 0.3217 - val_accuracy: 0.8640
Epoch 9/50
20/20 - 3s - loss: 0.3933 - accuracy: 0.8438 - val_loss: 0.2946 - val_accuracy: 0.8774
Epoch 10/50
20/20 - 3s - loss: 0.3654 - accuracy: 0.8344 - val_loss: 0.3012 - val_accuracy: 0.8694
Epoch 11/50
20/20 -

Build new model = VGG16 + Top model

In [112]:
from keras.applications.vgg16 import VGG16
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import keras
import tensorflow as tf
import os
import datetime


# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150
# keras.backend.set_image_dim_ordering('tf')
backend.image_data_format()

# creation of the base VGG pre-trained model
model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height,3))
print('Model loaded.')

#fetch the previously trained classifier
top_model = keras.models.load_model('bottleneck_fc_model')


# creation of a real model from vgg
new_model = Sequential()
for l in model.layers:
    new_model.add(l)


# concatenation of the base model with the top model
new_model.add(top_model)

new_model.summary()


Model loaded.
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 37, 

Fine tune new model

In [113]:
train_data_dir = 'Lab/data/DogCatChallenge/sampleDeep/train'
validation_data_dir = 'Lab/data/DogCatChallenge/sampleDeep/valid'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 10 #50
batch_size = 16

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)

###################
# for layer in new_model.layers[:15]:
#    layer.trainable = False
###################

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
new_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(learning_rate=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42)

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42)

# fine-tune the model

logdir = os.path.join("logs1", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# from keras.callbacks import ModelCheckpoint, EarlyStopping
# checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=3, save_best_only=True, save_weights_only=False, mode='max', save_freq=1)
# early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=3, mode='max')
# hist = model.fit_generator(steps_per_epoch=100,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

history = new_model.fit(
        train_generator,
        steps_per_epoch=20, #nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        batch_size = batch_size,
        ####### validation_steps=nb_validation_samples // batch_size,
        callbacks=[tensorboard_callback], #,checkpoint,early],
        verbose=2)

# for key in history.history:
#     print(key)

%reload_ext tensorboard
%tensorboard --logdir logs

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/10
20/20 - 119s - loss: 0.4497 - accuracy: 0.8375 - val_loss: 0.2408 - val_accuracy: 0.9187
Epoch 2/10
20/20 - 106s - loss: 0.2703 - accuracy: 0.8781 - val_loss: 0.1504 - val_accuracy: 0.9463
Epoch 3/10
20/20 - 108s - loss: 0.2097 - accuracy: 0.9125 - val_loss: 0.1294 - val_accuracy: 0.9450
Epoch 4/10
20/20 - 112s - loss: 0.1793 - accuracy: 0.9312 - val_loss: 0.1419 - val_accuracy: 0.9375
Epoch 5/10
20/20 - 109s - loss: 0.2116 - accuracy: 0.9250 - val_loss: 0.1460 - val_accuracy: 0.9400
Epoch 6/10
20/20 - 99s - loss: 0.1871 - accuracy: 0.9281 - val_loss: 0.1672 - val_accuracy: 0.9350
Epoch 7/10
20/20 - 74s - loss: 0.1998 - accuracy: 0.9062 - val_loss: 0.1549 - val_accuracy: 0.9375
Epoch 8/10
20/20 - 73s - loss: 0.1553 - accuracy: 0.9312 - val_loss: 0.1436 - val_accuracy: 0.9450
Epoch 9/10
20/20 - 73s - loss: 0.1163 - accuracy: 0.9625 - val_loss: 0.1618 - val_accuracy: 0.9475
Epoch 10/10
20/20 - 7

Reusing TensorBoard on port 6006 (pid 20508), started 11:32:17 ago. (Use '!kill 20508' to kill it.)

Prediction for a single sample

In [114]:
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
# from keras.applications.vgg16 import preprocess_input
# from keras.applications.vgg16 import decode_predictions
# from keras.applications.vgg16 import VGG16

# import os
# dirname = os.path.dirname("__file__")
# img_path = os.path.join(dirname, 'dogs-vs-cats/test1/test1/34.jpg') #dog

# # load an image from file
# image = load_img(img_path, target_size=(img_width, img_height))
# # convert the image pixels to a numpy array
# image = img_to_array(image)
# # reshape data for the model
# image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# # prepare the image for the VGG model
# image = preprocess_input(image)
# # predict the probability across all output classes
# yhat = new_model.predict(image) #[0][0]
# # pred = 0 if yhat<0.5 else 1
# # print(pred)
# label = 'Cat' if yhat == 0 else 'Dog'
# print(label)

Prediction for the whole test dataset

In [118]:
# test_data_dir = 'Lab/data/DogCatChallenge/sampleDeep/test_main'
test_data_dir = 'dogs-vs-cats-redux-kernels-edition/test'

test_datagen = ImageDataGenerator(rescale=1. / 255)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                        target_size=(img_width, img_height),
                                        class_mode=None,
                                        # classes=class_subset,
                                        batch_size=1,
                                        shuffle=False)

# from tf.keras.models import load_model

# final_model = load_model('saved_model/my_model')

vgg_preds = new_model.predict(testgen, verbose=1)
L = len(vgg_preds)
vgg_pred_list = vgg_preds.tolist()
full_pred = [vgg_pred_list[i][0] for i in range(L)]
print(len(full_pred)) 

Found 12500 images belonging to 1 classes.
12500/12500 [==============================] - 1238s 99ms/step
12500


In [119]:
import pandas as pd 

submission = pd.read_csv("dogs-vs-cats-redux-kernels-edition/sample_submission.csv")
submission['label'] = full_pred
submission.to_csv("full_sub.csv", index=False)

In [120]:
final_pred = [0 if x<0.5 else 1 for x in full_pred]
submission = pd.read_csv("dogs-vs-cats-redux-kernels-edition/sample_submission.csv")
submission['label'] = final_pred
submission.to_csv("full_sub1.csv", index=False)

Debugging... (prediction on new labeled images)

In [124]:
# test_data_dir = 'Lab/data/DogCatChallenge/sampleDeep/test_main'
test_data_dir = 'dogs-vs-cats-redux-kernels-edition/test_try'

test_datagen = ImageDataGenerator(rescale=1. / 255)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                        target_size=(img_width, img_height),
                                        class_mode=None,
                                        # classes=class_subset,
                                        batch_size=1,
                                        shuffle=False)

# from tf.keras.models import load_model

# final_model = load_model('saved_model/my_model')

vgg_preds_try = new_model.predict(testgen, verbose=1)
L = len(vgg_preds_try)
vgg_pred_list = vgg_preds_try.tolist()
full_pred = [vgg_pred_list[i][0] for i in range(L)]
final_pred = [0 if x<0.5 else 1 for x in full_pred]

Found 2000 images belonging to 1 classes.
2000/2000 [==============================] - 178s 89ms/step


In [125]:
from sklearn.metrics import log_loss, accuracy_score
labels = [0]*1000 + [1]*1000
print(len(final_pred))
print(log_loss(labels,final_pred))
print(accuracy_score(labels,final_pred))

2000
1.1743239946089754
0.966
